The basic task is to predict the price of a house depending on the features given. Results will be evaluated using RMSE.

In [95]:
#importing libraries

import pandas as pd

# Analysing data

In [151]:
df= pd.read_csv('./data/train.csv')

In [152]:
print(df.shape)
df.head()

(1460, 81)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [153]:
# from a quick look, it can be seen that there are many NaN values

# counting number of NaNs in all the columns

# as there are 81 columns, to see all the values use to_string() and sort_values() and to get a clearer picture

na_count = df.isna().sum().sort_values(ascending=False)

# printing columns with atleast 1 missing value
na_count = na_count[na_count>0]
print(na_count.to_string())

PoolQC          1453
MiscFeature     1406
Alley           1369
Fence           1179
FireplaceQu      690
LotFrontage      259
GarageYrBlt       81
GarageCond        81
GarageType        81
GarageFinish      81
GarageQual        81
BsmtFinType2      38
BsmtExposure      38
BsmtQual          37
BsmtCond          37
BsmtFinType1      37
MasVnrArea         8
MasVnrType         8
Electrical         1


In [154]:
# analyzing the columns and reading data description to see how to treat the missing values 
# print(df.LotFrontage.value_counts().to_string())
df.loc[df.LotFrontage.isna(),'LotFrontage'] = 0
df.loc[df.MasVnrArea.isna(),'MasVnrArea'] = 0

df.loc[df.FireplaceQu.isna(),'FireplaceQu'] = 'No_FP'
df.loc[df.Alley.isna(),'Alley'] = 'No_alley'
df.loc[df.PoolQC.isna(),'PoolQC'] = 'No_pool'
df.loc[df.MiscFeature.isna(),'MiscFeature'] = 'No_MF'
df.loc[df.Fence.isna(),'Fence'] = 'No_Fence'
df.loc[df.BsmtExposure.isna(),'BsmtExposure'] = 'No_Bment'
df.loc[df.BsmtQual.isna(),'BsmtQual'] = 'No_Bment'
df.loc[df.BsmtFinType2.isna(),'BsmtFinType2'] = 'No_Bment'
df.loc[df.BsmtFinType1.isna(),'BsmtFinType1'] = 'No_Bment'
df.loc[df.BsmtCond.isna(),'BsmtCond'] = 'No_Bment'

In [155]:
na_count_after = df.isna().sum().sort_values(ascending=False)
na_count_after = na_count_after[na_count_after>0]
print(na_count_after.to_string())

GarageFinish    81
GarageYrBlt     81
GarageType      81
GarageQual      81
GarageCond      81
MasVnrType       8
Electrical       1


In [157]:
# checking if garage attributes are missing for same obersvations

_ = df[df.GarageFinish.isna()].loc[:,['GarageFinish','GarageYrBlt','GarageQual','GarageCond','GarageType']]
print(_.to_string())

     GarageFinish  GarageYrBlt GarageQual GarageCond GarageType
39            NaN          NaN        NaN        NaN        NaN
48            NaN          NaN        NaN        NaN        NaN
78            NaN          NaN        NaN        NaN        NaN
88            NaN          NaN        NaN        NaN        NaN
89            NaN          NaN        NaN        NaN        NaN
99            NaN          NaN        NaN        NaN        NaN
108           NaN          NaN        NaN        NaN        NaN
125           NaN          NaN        NaN        NaN        NaN
127           NaN          NaN        NaN        NaN        NaN
140           NaN          NaN        NaN        NaN        NaN
148           NaN          NaN        NaN        NaN        NaN
155           NaN          NaN        NaN        NaN        NaN
163           NaN          NaN        NaN        NaN        NaN
165           NaN          NaN        NaN        NaN        NaN
198           NaN          NaN        Na

In [158]:
# as the garage features are missing for the observations where garage is not present, we can impute them all with same value
df.loc[df.GarageFinish.isna(),'GarageFinish'] = 'No_Garage'
df.loc[df.GarageYrBlt.isna(),'GarageYrBlt'] = 'No_Garage'
df.loc[df.GarageQual.isna(),'GarageQual'] = 'No_Garage'
df.loc[df.GarageCond.isna(),'GarageCond'] = 'No_Garage'
df.loc[df.GarageType .isna(),'GarageType'] = 'No_Garage'

na_count_after = df.isna().sum().sort_values(ascending=False)
na_count_after = na_count_after[na_count_after>0]
print(na_count_after.to_string())

MasVnrType    8
Electrical    1


In [159]:
# for MasVnrType and Electrical, there is no clear path to what can be done. 
# Therefore, I will use KNNImputer 
# I could have filled them with most frequent element too. However, I think KNNImputer works the best
